In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import Analysis
import pylab
import regression_cv as rcv
import ipywidgets
import FitRoutine2 as fr
import itertools
%matplotlib 

Using matplotlib backend: TkAgg


In [2]:
%matplotlib 

Using matplotlib backend: TkAgg


In [3]:
data_hold = list()

In [5]:
height = 11.4
amp = 0.3
period = 9

phase_selection = [0.66]

num_slips = 1

noise_amp = 0.03
random_seed = 82


freq = (2 * np.pi)/period
x = np.linspace(-30,-5,10000)
p = list()
hold = 0
phase = list()
phase_hold = 0
is_phase = False

for i in range(len(x)):
    if np.random.random()<float(num_slips)/len(x):
        #hold = (hold + 1)%2
        if is_phase:
            phase_hold = 0
            is_phase = False
        else:
            phase_hold = phase_selection[np.random.randint(0,len(phase_selection))]
            is_phase = True
    #p.append(hold)
    phase.append(phase_hold)
p = np.array(phase)*np.pi #np.array(p)

y =  height + amp * np.sin(freq * x + p) + noise_amp*np.random.normal(size=len(x))

In [4]:
plt.rc('text', usetex=True) 

# Helvetica 
# set serif, too. Otherwise setting to times and then 
# Helvetica causes an error. 
font = {'family':'sans-serif','sans-serif':['Helvetica'], 
        'serif':['cm10']} 
preamble  = r""" 
               \usepackage{color} 
               \usepackage[tx]{sfmath} 
               \usepackage{helvet} 
            """ 


# Unicode for Tex 
#preamble =  r"""\usepackage[utf8]{inputenc}""" + preamble 
# inputenc should be set automatically 
plt.rcParams['text.latex.unicode']=True 

#print font, preamble 
plt.rc('font',**font) 
plt.rcParams['text.latex.preamble'] = preamble
mpl.rcParams.update({'font.size': 11})

In [6]:
plt.figure()
plt.plot(x,y)

In [14]:
x_fit = np.linspace(-30,-24,1000)
y_fit =  height + amp * np.sin(freq * x_fit)
plt.plot(x_fit,y_fit,'k',lw=4)
y_fit =  height + amp * np.sin(freq * x_fit+0.66*np.pi)
plt.plot(x_fit,y_fit,'r--',lw=4)




x_fit = np.linspace(-24,-14.1734,1000)
y_fit =  height + amp * np.sin(freq * x_fit)
plt.plot(x_fit,y_fit,'k--',lw=4)
y_fit =  height + amp * np.sin(freq * x_fit+0.66*np.pi)
plt.plot(x_fit,y_fit,'r',lw=4)



In [19]:
plt.xlabel('$V_P(mV)$',size=18)
plt.ylabel('$R_D(k\Omega)$',size=18)

plt.xlim(-30,-18)
plt.ylim(11,11.8)

plt.xticks([-27,-23,-19])
plt.yticks([11.1,11.4,11.7])

plt.axvline(-24,lw=3,c='k')

In [20]:
plt.savefig('PhaseSlipExample',dpi=300)

In [419]:
#Adding large noise spikes:
#Upper bounds on params
large_noise_length = 10
large_noise_events = 100
large_noise_amp = 0.2

noise_y = np.array(y)

i = 0
while i < len(noise_y):
    if np.random.random()<float(large_noise_events)/len(noise_y):
        amp = np.random.normal()*large_noise_amp
        for j in range(np.random.randint(0,large_noise_length)):
            noise_y[i+j] += amp
        i += j
    else:
        i += 1


In [420]:
plt.figure()
plt.plot(x,noise_y)

In [421]:
sweep = Analysis.Sweep([x,noise_y],1.0,[])

In [422]:
data_hold.append(sweep)

In [423]:
len(data_hold)

7

In [424]:
np.save('Fake Data Hold Multi Phase',data_hold)

In [8]:
reg3 = rcv.run_regression(data_hold, rcv.regression_CV, d = 3, residual = rcv.residual_new, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-3], loss = 'huber')
#reg = rcv.run_regression(data_hold, rcv.regression_CV, d = 5, residual = rcv.residual_new, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-3], loss = 'huber')

Running sweep 0
Running sweep 1
<type 'exceptions.KeyboardInterrupt'>
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6


In [6]:
def search(seq, t):  
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max <= min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m


def plot(data,result,full=False,if_data=True):
    x_min = result['x_min']
    x_max = result['x_max']
    if full:
        x = data.x
        y = data.y
    else:
        i_min = search(data.x,x_min)
        i_max = search(data.x,x_max)+1

        x = data.x[i_min:i_max]
        y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    params = result['result'].x
    model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
    if if_data:
        plt.plot(x,y)
    plt.plot(x,model_r,c='r')
    plt.plot(x,model_l,c='k')

In [4]:
reg = np.load('Fake Data Reg Multi Phase.npy')
data_hold = np.load('Fake Data Hold Multi Phase.npy')

In [51]:
val_acc = 0.6
filenum = 1


count = 0
for i in reg3[filenum]:
    if i['result'] is not None:
        if i['val_acc']>val_acc and 0.17 < i['result'].x[4] < 1.80:
            plt.figure()
            plot(data_hold[filenum],i)
            plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4])+ ', '+ str(i['val_acc']))
    count += 1

In [12]:
bf_hold = list()

In [28]:
n = [47,27,11]
temp = list()
for i in n:
    temp.append(reg3[filenum][i])
bf_hold.append(temp)

In [29]:
len(bf_hold)

7

In [224]:
np.save('Fake Data Reg3 Multi Phase',reg3)

In [ ]:
val_acc = 0.50
filenum = 1
hold = list()
count = 0
for i in reg[filenum]:
    if i['result'] is not None:
        if i['val_acc']>val_acc and 0.16 < i['result'].x[4] < 1.80:
            if i['result'].x[4] > 1.0:
                hold.append(2-i['result'].x[4])
            else:
                hold.append(i['result'].x[4])
    count += 1
print(np.mean(hold))

In [ ]:
len(bf_hold)

In [30]:
def csf(scipy_fit):
    x = scipy_fit['result'].x
    out = list()
    out.append(x[2])
    out.append(x[1])
    out.append(x[3]+x[4]*np.pi)
    out.append(x[3])
    out.append(x[0])
    out.append(x[5])
    out.append(scipy_fit['val_acc'])
    out.append(scipy_fit['x_min'])
    out.append(scipy_fit['x_max'])
    phi = x[4]
    out.append(phi)
    out.append(scipy_fit['ci_deltaphi'])
    out.append((scipy_fit['x_min'] + scipy_fit['x_max'])/2)
    return np.array(out)

In [31]:
fit_hold = list()
for i in bf_hold:
    temp = list()
    for j in i:
        temp.append(csf(j))
    fit_hold.append(temp)

In [32]:
len(fit_hold)

7

In [33]:
np.save('Fake Data BF Multi Phase', fit_hold)

In [123]:
fake = Analysis.Run(np.load("Fake Data Multi Phase.npy"), np.load("Fake Data Multi Phase.npy"), np.ones(10),np.load('Fake Data BF Multi Phase.npy'))

In [140]:
se = list()

In [141]:
mom = 5
mindiff = 0.03
dV = 0.1
minsep = 1.0

In [281]:
filenum = 0

In [282]:
plt.figure()
fake.prune_data[filenum].plot()
for i in fake.prune_data[filenum].fits:
    i.plot(False)

In [283]:
plt.xlabel('V$_P$ (mV)')
plt.ylabel('R$_D$ (k$\Omega$)')

In [275]:
plt.rc('font', size=12)    

In [284]:
plt.savefig('Fake Data Multi Phase Full Sweep_1',dpi=1000)

In [285]:
plt.clf()

In [233]:
x = fake.prune_data[filenum].x
y = fake.prune_data[filenum].y
fit_hold = fake.prune_data[filenum].fits

In [234]:
temp = list(fr.countEdge(x,y,fit_hold,1,dV=dV,momentum=mom,minsep=minsep,mindiff = mindiff))

In [235]:
temp

[array([-9.61921192,  1.        ,  0.94763189,  0.79520697]),
 array([-8.87663766,  1.        ,  0.94763189,  0.79520697]),
 array([-5.78882888,  1.        ,  0.94763189,  0.79520697]),
 array([-11.04435444,   1.        ,   1.22291418,   0.75913978]),
 array([-17.6750175 ,   1.        ,   1.26853338,   0.81136364]),
 array([-17.4349935 ,   1.        ,   1.26853338,   0.81136364]),
 array([-14.90474047,   1.        ,   1.26853338,   0.81136364]),
 array([-14.16966697,   1.        ,   1.26853338,   0.81136364]),
 array([-19.81523152,   1.        ,   0.74177097,   0.81086957]),
 array([-27.11846185,   1.        ,   1.78879941,   0.8091954 ]),
 array([-23.64311431,   1.        ,   1.78879941,   0.8091954 ])]

In [200]:
plt.figure()
plt.plot(x,y)
for i in temp:
    j = i[0]
    index = rcv.search(x,j)
    plt.plot(x[index],y[index],'y*',markersize=20)

In [201]:
se.append(np.array(temp))

In [202]:
len(se)

7

In [203]:
np.save('Fake Data SE Multi Phase', se)

In [183]:
del temp[0]

In [43]:
c = 0
for i in fe:
    if len(i)>0:
        plt.figure()
        plt.plot(x,y)
        fit_hold[c].plot(False)
        for j in i:
            index = rcv.search(x,j[3])
            plt.plot(x[index],y[index],'c*',markersize=20)
            index = rcv.search(x,j[2])
            plt.plot(x[index],y[index],'y*',markersize=20)
    c += 1

In [18]:
sof = fr.smoothOnFit(x,y,fit_hold,mindiff=0.03)

In [27]:
for i in sof:
    plt.figure()
    plt.plot(i[:,2],i[:,1])

In [22]:
pof = fr.pruneOnFit(x,y,fit_hold,mindiff=0.02)

In [23]:
for i in pof:
    plt.figure()
    plt.plot(i[:,2],i[:,1])

In [16]:
sof[0][1000]

array([ -1.        ,   0.        , -27.44974497,   0.        ])

In [55]:
plt.figure()
plt.plot(x,y)
fs = fr.finalSlips(x,y,fit_hold,dV=0.09,mindiff=0.04)
for j in fs:
    for i in j:
        index = rcv.search(x,(i[2]+i[3])/2)
        plt.plot(x[index],y[index],'c*',markersize=20)


In [57]:
fs

array([ array([[  1.0197071 ,   2.        , -22.25672567, -22.25422542,
          0.80687831]]),
       array([[  0.98482398,   1.        , -20.88158816, -20.87908791,
          0.81201044],
       [  0.98482398,   2.        , -20.3540354 , -20.35153515,
          0.81201044],
       [  0.98482398,   1.        , -17.19621962, -17.19371937,
          0.81201044]]),
       array([[ 1.03834573,  2.        , -8.73537354, -8.73287329,  0.77950713]])], dtype=object)

In [ ]:
hold = list()
dV = 0.1
n = 0
for i in e:
    for j in range(len(i)-1):
        if i[j][1] != i[j+1][1] and i[j+1][4] > dV and i[j][4] > dV:
            hold.append([(i[j][3]+i[j+1][2])/2,i[j][0],i[j][5],n])
            n+=1

In [ ]:
plt.figure()
plt.plot(x,y)
for j in hold:
    index = rcv.search(x,j[0])
    plt.plot(x[index],y[index],'y*',markersize=20)

In [56]:
all_slips = list()
dV = 0.09
for filenum in range(len(fake.prune_data)):
    x = fake.prune_data[filenum].x
    y = fake.prune_data[filenum].y
    fit_hold = fake.prune_data[filenum].fits
    e = fr.findEdge(x,y,fit_hold,dV=dV, mindiff=0.10)
    hold = list()
    n = 0
    for i in e:
        for j in range(len(i)-1):
            if i[j][1] != i[j+1][1] and i[j+1][4] > dV and i[j][4] > dV:
                hold.append([(i[j][3]+i[j+1][2])/2,i[j][0],i[j][5],n])
                n+=1
    del_hold = list()
    for a,b in itertools.combinations(hold, 2):
        if abs(a[0]-b[0]) < dV:
            if a[2]>b[2]:
                del_hold.append(b[3])
            else:
                del_hold.append(a[3])
    del_hold = sorted(set(del_hold))
    for i in reversed(del_hold):
        del hold[i]
    all_slips.append(hold)
    plt.figure()
    plt.plot(x,y)
    for j in hold:
        index = rcv.search(x,j[0])
        plt.plot(x[index],y[index],'y*',markersize=20)

In [ ]:
for filenum in range(len(fake.prune_data)):
    plt.figure()
    fake.prune_data[filenum].plot()
    x = fake.prune_data[filenum].x
    y = fake.prune_data[filenum].y
    for j in all_slips[filenum]:
        index = rcv.search(x,j[0])
        plt.plot(x[index],y[index],'y*',markersize=20)


In [74]:
np.save('Fake Slips Noise Spikes',all_slips)

In [134]:
L = len(fake.prune_data)
m = 0
for i in fake.prune_data:
    if len(i.fits) > m:
        m = len(i.fits)

        
m += 2 # one for fits, one for hist
total = L*m
c = 1
n = 0

phase_hold = [0.25,0.33,0.66,1,1.25]

plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig = plt.figure(figsize=(20,40))
for i in fake.prune_data:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([min(i.x),max(i.x)])
        c += 1
        temp += 1
        
    ax = plt.subplot(L,m,c)
    x = fake.prune_data[n].x
    y = fake.prune_data[n].y
    fake.prune_data[n].plot()
    for j in all_slips[n]:
        index = rcv.search(x,j[0])
        plt.plot(x[index],y[index],'y*',markersize=7)
    c += 1
    temp += 1
    
    ax = plt.subplot(L,m,c)
    plt.hist(all_slips[n][:,1], bins=bins)
    plt.xlim([0,2.0])
    plt.xticks([0,0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0])
    ax.annotate("{}$\pi$ Model Value".format(phase_hold[n]), xy=(0.72, 0.80), xycoords="axes fraction")
    c += (m-temp)
    n += 1
fig.tight_layout()

NameError: name 'all_slips' is not defined

In [ ]:
for i in range(len(all_slips)):
    for j in range(len(all_slips[i])):
        if all_slips[i][j][1] > 1.3:
            all_slips[i][j][1] = 2 - all_slips[i][j][1]
    

In [77]:
fig.savefig('Fake Data Noise Spikes Full Algo',dpi=1000)

In [66]:
all_slips = list()
for filenum in range(5):

    x = fake.prune_data[filenum].x
    y = fake.prune_data[filenum].y
    fit_hold = fake.prune_data[filenum].fits
    fs = fr.finalSlips(x,y,fit_hold,dV=0.09,mindiff=0.04)
    hold = list()
    for j in fs:
        for i in j:
            hold.append([(i[2]+i[3])/2.0,i[0]])
    all_slips.append(np.array(hold))

In [121]:
data = list()
for filenum in range(7):
    data.append(np.array([data_hold[filenum].x,data_hold[filenum].y]))

In [122]:
np.save('Fake Data Multi Phase', data)

In [269]:
#Plot with histogram and edges:

run = Analysis.Run(np.load("Fake Data Multi Phase.npy"), np.load("Fake Data Multi Phase.npy"), np.ones(10),np.load('Fake Data BF Multi Phase.npy'))
bf = np.load('Fake Data BF Multi Phase.npy')
se = np.load('Fake Data SE Multi Phase.npy')

m = 0
for i in [bf[0],bf[3]]:
    if len(i) > m:
        m = len(i)
m += 1 #for histo


L = len([bf[0],bf[3]])
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig = plt.figure(figsize=(20,5))
for i in [run.prune_data[0],run.prune_data[3]]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([j.pos_left,j.pos_right])
        if len(se[n]) > 0:
            for k in se[n][np.abs(se[n][:,2] - bf[n][temp][9])<0.01]:
                ax.axvline(k[0],color='k',lw=3)
        c += 1
        temp += 1
    ax = plt.subplot(L,m,c)
    if len(se[n]) > 0:
        plt.hist(se[n][:,2], bins=bins)
        plt.xlim([0,2.0])
        plt.xticks([0,0.5,1.0,1.5,2.0])
    c += (m-temp)
    n = 3 #Changed
fig.tight_layout()


In [270]:
fig.savefig('Fake Data Multi Phase',dpi=1000)

In [207]:
se = np.load('Fake Data SE Multi Phase.npy')

In [211]:
se[0][:,2]

array([ 1.77830088,  1.77830088,  1.07994229,  0.97029265,  1.75305926,
        1.75305926])

In [216]:
se[0][np.abs(se[0][:,2] - bf[0][0,9])<0.01]

TypeError: list indices must be integers, not tuple

In [217]:
np.abs(se[0][:,2] - bf[0][0,9])<0.01

TypeError: list indices must be integers, not tuple

In [220]:
bf[0][0][9]

1.7783008758174907

In [289]:
x_1 = np.linspace(0,1,100)
x_2 = np.linspace(1,2,100)
y_1 = np.sin(x_1*(2*np.pi))
y_1_e = np.sin(x_2*(2*np.pi))
y_2 = np.sin(x_2*(2*np.pi)-1.25*np.pi)
y_2_e = np.sin(x_1*(2*np.pi)-1.25*np.pi)
y_3 = np.sin(x_2*(2*np.pi)+0.75*np.pi)
y_3_e = np.sin(x_1*(2*np.pi)+0.75*np.pi)

In [293]:
plt.figure()
plt.plot(x_1,y_1,'k')
plt.plot(x_2,y_1_e,'k--')
plt.plot(x_2,y_2,'r',lw=3)
plt.plot(x_1,y_2_e,'r--',lw=3)
plt.plot(x_2,y_3,'y')
plt.plot(x_1,y_3_e,'y--')

In [294]:
run.prune_data[0].plot()

In [295]:
plt.figure()
run.prune_data[0].plot()
for i in run.prune_data[0].fits:
    i.plot(False)

In [297]:
plt.figure()
run.prune_data[0].plot()
run.prune_data[0].plot_fit(-1)

In [302]:
run.prune_data[0].plot()
plt.xlabel('V$_P$ (mV)')
plt.ylabel('R$_D$ (k$\Omega$)')
plt.tight_layout()

In [301]:
plt.savefig('Fake Data Sweep',dpi=1000)

In [309]:
run.prune_data[0].plot()
plt.xlabel('V$_P$ (mV)')
plt.ylabel('R$_D$ (k$\Omega$)')
i = run.prune_data[0].fits[-1]
p = -22.0048
#i.plot_select('l','k',3,p-4,p)
#i.plot_select('l','k',3,p,p+4,'--')
#i.plot_select('r','r',3,p-4,p,'--')
#i.plot_select('r','r',3,p,p+4)
plt.xlim(p-4,p+4)
plt.ylim(10.986,11.8468)
plt.tight_layout()